# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("..\\WeatherPy\\output data\\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,visby,57.6409,18.2960,4.12,70,100,6.69,SE,1742259566
1,1,waitangi,-43.9535,-176.5597,16.48,96,48,9.67,NZ,1742259568
2,2,berdigestyakh,62.0989,126.6992,-9.69,93,99,2.09,RU,1742259569
3,3,keflavik,64.0049,-22.5624,7.24,100,100,6.69,IS,1742259570
4,4,acapulco de juarez,16.8634,-99.8901,23.44,74,0,3.70,MX,1742259571


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_1= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    hover_cols=["City", "Humidity"],
    color="City"
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_cities_df=city_data_df[
    (city_data_df["Max Temp"]<34) &
    (city_data_df["Max Temp"]>6) &
    (city_data_df["Wind Speed"]<3) &
    (city_data_df["Cloudiness"]<=50)
].dropna()

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
narrow_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,tamanrasset,22.7850,5.5228,19.95,17,0,2.57,DZ,1742259580
13,13,adamstown,-25.0660,-130.1015,25.44,70,0,1.41,PN,1742259582
17,17,cudillero,43.5622,-6.1459,8.51,80,0,2.06,ES,1742259586
19,19,hawaiian paradise park,19.5933,-154.9731,26.60,74,34,1.79,US,1742259589
20,20,bel ombre,-4.6167,55.4167,27.91,83,20,2.06,SC,1742259590
...,...,...,...,...,...,...,...,...,...,...
567,567,yanqul,23.5905,56.5501,17.40,47,0,0.87,OM,1742260242
568,568,marka,1.7159,44.7717,25.67,80,12,2.51,SO,1742260243
570,570,piriapolis,-34.8682,-55.2744,20.09,83,0,2.57,UY,1742260245
575,575,sathupalli,17.2497,80.8690,21.44,92,18,2.59,IN,1742260022


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= narrow_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,tamanrasset,DZ,22.7850,5.5228,17,
13,adamstown,PN,-25.0660,-130.1015,70,
17,cudillero,ES,43.5622,-6.1459,80,
19,hawaiian paradise park,US,19.5933,-154.9731,74,
20,bel ombre,SC,-4.6167,55.4167,83,
...,...,...,...,...,...,...
567,yanqul,OM,23.5905,56.5501,47,
568,marka,SO,1.7159,44.7717,80,
570,piriapolis,UY,-34.8682,-55.2744,83,
575,sathupalli,IN,17.2497,80.8690,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row["Lat"]
    lng=row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
   

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tamanrasset - nearest hotel: فندق الأمان
adamstown - nearest hotel: No hotel found
cudillero - nearest hotel: Sol de la Blanca
hawaiian paradise park - nearest hotel: No hotel found
bel ombre - nearest hotel: Villa La Cachette
pointe-noire - nearest hotel: HÔTEL F.A.R.E
margaret river - nearest hotel: Margaret River Hotel
itamaraju - nearest hotel: Hotel Vista Bela
kindu - nearest hotel: Hôtel Karibu
miles city - nearest hotel: Historic Olive Hotel
illizi - nearest hotel: بوناقة
kuta - nearest hotel: Ida Hotel Bali
remire-montjoly - nearest hotel: Complexe Belova
coquimbo - nearest hotel: Hotel Iberia
tura - nearest hotel: Hotel Polo Orchid
howrah - nearest hotel: Sun India Guest House
pocone - nearest hotel: Andirah
codrington - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
argo - nearest hotel: Apollon Hotel
mar del plata - nearest hotel: Hotel 3 de Abril
ancud - nearest hotel: Cabañas
khanpur mahar - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
12,tamanrasset,DZ,22.7850,5.5228,17,فندق الأمان
13,adamstown,PN,-25.0660,-130.1015,70,No hotel found
17,cudillero,ES,43.5622,-6.1459,80,Sol de la Blanca
19,hawaiian paradise park,US,19.5933,-154.9731,74,No hotel found
20,bel ombre,SC,-4.6167,55.4167,83,Villa La Cachette
...,...,...,...,...,...,...
567,yanqul,OM,23.5905,56.5501,47,No hotel in this town
568,marka,SO,1.7159,44.7717,80,Salsabiil Hotel
570,piriapolis,UY,-34.8682,-55.2744,83,Esmeralda
575,sathupalli,IN,17.2497,80.8690,92,Kalpatarru lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    hover_cols= ['Hotel Name','country'],
    color="Hotel Name"
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity)